In [7]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# from quantile_forest import RandomForestQuantileRegressor

from is_holiday import check_holiday
from utilites import hour_features, day_features, dayofyear_features

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib

In [1]:
path = 'data/hystreet_fussgaengerfrequenzen_seit2021.csv'
df = load_and_preprocess(path)
df = df.loc[df['location_id'] == 330]

df

NameError: name 'load_and_preprocess' is not defined

In [2]:
%matplotlib tk
plt.plot(pd.to_datetime(df['date']), df['pedestrians_count'])
plt.show()

NameError: name 'plt' is not defined

In [6]:

matplotlib.use("TkAgg")
plt.figure()
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_seen['date']], df_seen['pedestrians_count'])
plt.show()

plt.figure()
plt.hist(df_seen['pedestrians_count'], bins=50)
plt.show()

window_size = 24*365  # You can adjust this as needed
m_avg = df_seen['pedestrians_count'].rolling(window=window_size, center=True).mean()

# Plot original data and centered moving average
plt.figure()
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_seen['date']], df_seen['pedestrians_count'], label='Pedestrians Count')
plt.plot([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_seen['date']], m_avg, label='Moving Average', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Pedestrians Count')
plt.title('Pedestrians Count with Centered Moving Average')
plt.legend()
plt.show()

df_seen = df_seen.drop(columns='location_id')

# Temporary drop date
df_seen['holiday'] = df_seen.apply(lambda row: check_holiday(row['date']), axis = 1)

df_seen['time_cos'] = df_seen.apply(lambda row: hour_features(row['date'])[0], axis = 1)
df_seen['time_sin'] = df_seen.apply(lambda row: hour_features(row['date'])[1], axis = 1)

df_seen['day_cos'] = df_seen.apply(lambda row: day_features(row['date'])[0], axis = 1)
df_seen['day_sin'] = df_seen.apply(lambda row: day_features(row['date'])[1], axis = 1)

df_seen['dayofyear_cos'] = df_seen.apply(lambda row: dayofyear_features(row['date'])[0], axis = 1)
df_seen['dayofyear_sin'] = df_seen.apply(lambda row: dayofyear_features(row['date'])[1], axis = 1)

df_unseen = df_seen[[datetime.strptime(d, '%Y-%m-%d %H:%M:%S') <= datetime(2022, 5, 5) for d in df_seen['date']]]
df_seen = df_seen[[datetime.strptime(d, '%Y-%m-%d %H:%M:%S') > datetime(2022, 5, 5) for d in df_seen['date']]]
df_mid_old = df_seen.copy()

df_seen = df_seen.drop(columns='date')
# df_mid['date'][0].split()
# 
df_seen

NameError: name 'df_seen' is not defined

In [24]:
train, test = train_test_split(df_seen, test_size=0.2)

In [23]:
forest = RandomForestQuantileRegressor(verbose=True, oob_score=True, n_estimators=300, random_state=42)
forest.fit(train.drop(columns='pedestrians_count'), train['pedestrians_count'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   25.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished


RandomForestQuantileRegressor(n_estimators=300, oob_score=True, random_state=42,
                              verbose=True)

In [27]:
forest.score(test.drop(columns='pedestrians_count'), test['pedestrians_count'])

C:\Users\cemke\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished


0.7059317320345434

In [28]:
forest.score(df_unseen.drop(columns=['pedestrians_count', 'date']), df_unseen['pedestrians_count'])

C:\Users\cemke\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished


0.7212427733243196

In [29]:
reals = np.array(df_unseen['pedestrians_count'])
preds = np.array(forest.predict(df_unseen.drop(columns=['pedestrians_count', 'date']),quantiles=[0.5]))
preds_up = np.array(forest.predict(df_unseen.drop(columns=['pedestrians_count', 'date']),quantiles=[0.95]))
preds_down = np.array(forest.predict(df_unseen.drop(columns=['pedestrians_count', 'date']),quantiles=[0.05]))
dates = np.array([datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df_unseen['date']])

C:\Users\cemke\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
C:\Users\cemke\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished
C:\Users\cemke\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestQuantileRegressor was fitted without feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.2s finished


In [30]:
%matplotlib tk
matplotlib.use("TkAgg")
plt.figure()
plt.plot(dates, reals, "r")
#plt.plot(dates, preds, "k")
plt.plot(dates, preds_up, "k--")
plt.plot(dates, preds_down, "k--")
plt.show()

In [13]:
forest.oob_prediction_

array([ 399.36842105,  120.19444444,  654.46341463, ..., 1894.25      ,
        979.97435897, 1532.94444444])